This notebook goes through the process of building a Json agent from scratch. [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/synacktraa/ai-agents/blob/main/built-from-scratch/Json-agent.ipynb)

**Tutorial by James Murdza**: 
<a href="https://youtu.be/xs5jTcv-2zY">
    <img src="https://img.shields.io/badge/YouTube-FF0000?style=for-the-badge&logo=youtube&logoColor=white" alt="YouTube" width=90px height=20px>
</a>


In [ ]:
%%capture --no-stderr
%pip install "docstring-parser>=0.16" duckduckgo-search jsonref litellm loguru pydantic requests

## Implementing a `JsonAgent` compatible tool

Why do we need it in the first place?

`Function calling` introduces powerful capabilities to interact with AI models, but manually creating and maintaining function schemas is a complex and error-prone process. We often struggle with:

- Keeping schemas synchronized with code changes
- Ensuring type safety and input validation
- Supporting multiple AI platform formats
- Reducing repetitive boilerplate code

It will be responsible for:

- Schema generation in all known function calling formats (`OpenAI`, `Anthropic`, and `Gemini`)
- Extracting docstring metadata for populating schema
- Validating python or JsON input against the function's parameter specification
- Catching all exception raised by the associated function and returning `ExceptionInfo` instead of just halting the execution. The exception info can be used by agent to understand what went wrong and how to fix it. This also helps in when defining the function, as we don't have to explicitly catch and handle exceptions.

In [75]:
from dataclasses import dataclass
from inspect import isfunction
from typing import (
    Any, 
    Callable, 
    Generic, 
    get_args, 
    get_origin, 
    get_type_hints,
    Optional, 
    TypeVar, 
    ParamSpec, 
    Protocol
)

from docstring_parser import parse, Docstring
from jsonref import replace_refs
from pydantic import BaseModel, TypeAdapter 


P_Spec = ParamSpec("P_Spec")
"""Type to represent parameter specification of a function"""

class SupportsStr(Protocol):
    def __str__(self) -> str: ...

T_Retval = TypeVar("T_Retval", bound=SupportsStr)
"""Type representing the return value of the function that can be converted to string."""


@dataclass
class ExceptionInfo:
    """Information about an exception."""
    name: str
    message: str


class JsonTool(Generic[P_Spec, T_Retval]):
    """
    Class for creating Json agent compatible tool from a function.
    
    It will be responsible for:

    - Schema generation in all known function calling formats (`OpenAI`, `Anthropic`, and `Gemini`)
    - Extracting docstring metadata for populating schema
    - Validating python or JsON input against the function's parameter specification
    - Catching all exception raised by the associated function and returning `ExceptionInfo` instead of just halting the execution. The exception info can be used by agent to understand what went wrong and how to fix it.
    """
    def __init__(self, __fn: Callable[P_Spec, T_Retval]) -> None:
        if not isfunction(__fn):
            raise TypeError("Expected a function object")

        self.__docstring = parse(__fn.__doc__ or "")
        if self.__docstring.description is None:
            raise ValueError("Function must include a description")

        self.__adapter = TypeAdapter(__fn)

        self.fn = __fn
        self.name = self.fn.__name__
        self.description = self.__docstring.description


    def __call__(self, *args: Any, **kwargs: Any) -> T_Retval:
        """Call the function association with the adapter."""
        return self.fn(*args, **kwargs)

    @property
    def standard_schema(self) -> dict[str, Any]:
        """Function schema in standard format."""
        pydantic_schema = self.__adapter.json_schema()
        if "$defs" in pydantic_schema:
            pydantic_schema = replace_refs(pydantic_schema, lazy_load=False)
            _ = pydantic_schema.pop("$defs", None)
        return update_object_schema(self.fn, pydantic_schema, self.__docstring)

    @property
    def openai_schema(self) -> dict[str, Any]:
        """Function schema in OpenAI function calling format. Compatible with almost all models."""
        return to_openai_function_calling_format(self.name, self.standard_schema)

    @property
    def anthropic_schema(self) -> dict[str, Any]:
        """Function schema in Anthropic function calling format."""
        return to_anthropic_function_calling_format(self.name, self.standard_schema)

    @property
    def gemini_schema(self) -> dict[str, Any]:
        """Function schema in Gemini function calling format."""
        return to_gemini_function_calling_format(self.name, self.standard_schema)

    def call_with_raw_arguments(
        self, arguments: dict[str, Any] | str, *, strict: bool | None = None, **kwargs: Any
    ) -> T_Retval | ExceptionInfo:
        """
        Validate the raw arguments against the function parameter specification and get output.

        Args:
            arguments: The dictionary object or JSON string to validate.
            strict: Whether to perform strict validation.
            kwargs: Additional keyword arguments to pass to the pydantic.TypeAdapter's validate method.
        """
        try:
            if isinstance(arguments, str):
                return self.__adapter.validate_json(arguments, strict=strict, **kwargs)
            return self.__adapter.validate_python(arguments, strict=strict, **kwargs)
        except Exception as e:
            return ExceptionInfo(name=type(e).__name__, message=str(e))

def tool(__fn: Callable[P_Spec, T_Retval]) -> JsonTool[P_Spec, T_Retval]:
    """Decorate a function to create a tool adapter"""
    return JsonTool(__fn)


# ----------- Some utilities to clean and transform function schema ---------- #


def update_object_schema(__type: type, schema: dict[str, Any], docstring: Docstring) -> dict[str, Any]:
    """
    Update object type schema with its type's docstring

    Args:
        __type:  The type to update schema for
        schema:  The schema to update
        docstring:  The docstring to use for updating

    Returns:
        The updated schema
    """
    if desc := docstring.description or schema.get("description"):
        schema["description"] = desc
    
    typehints = get_type_hints(__type)
    for p_name, p_schema in schema["properties"].items():
        if p_name in typehints:
            schema["properties"][p_name] = update_schema(typehints[p_name], p_schema, docstring)

        # Populate parameter description from docstring
        param = next((p for p in docstring.params if p.arg_name == p_name), None)
        if param and (p_desc := param.description or p_schema.get("description")):
            schema["properties"][p_name]["description"] = p_desc
    
    return schema


def update_array_schema(__type: type, schema: dict[str, Any], docstring: Docstring) -> dict[str, Any]:
    """
    Update array type schema items/prefixItems with its type's docstring

    Args:
        __type:  The type to update schema for
        schema:  The schema to update
        docstring:  The docstring to use for updating

    Returns:
        The updated schema
    """
    if "items" in schema:
        schema["items"] = update_schema(__type, schema["items"], docstring)

    elif all("title" in item for item in schema.get("prefixItems", [])):
        # we assume it's named tuple
        prefix_items = []
        p_name_to_desc = {p.arg_name: p.description for p in docstring.params}
        for p_schema in schema["prefixItems"]:
            if (p_name := p_schema["title"].lower()) in p_name_to_desc:
                p_schema["description"] = p_name_to_desc[p_name]
            prefix_items.append(p_schema)
        schema["prefixItems"] = prefix_items
    
    return schema


def update_schema(__type: type, schema: dict[str, Any], docstring: Docstring) -> dict[str, Any]:
    """
    Update type schema

    Args:
        __type:  The type to update schema for
        schema:  The schema to update
        docstring:  The docstring to use for updating

    Returns:
        The updated schema
    """
    if "anyOf" in schema:
        of_key = "anyOf"
    elif "oneOf" in schema:
        of_key = "oneOf"
    else:
        of_key = None

    origin, arg_types = get_origin(__type) or __type, get_args(__type)
    if of_key is not None:
        schema[of_key] = list(
            update_schema(_type, _schema, docstring) 
            for _type, _schema in zip(arg_types, schema[of_key])
        )

    elif schema.get("type") == "object" and "properties" in schema:
        if not schema.get("additionalProperties", True):
            _ = schema.pop("additionalProperties")
        schema = update_object_schema(origin, schema, docstring)
    
    elif schema.get("type") == "array":
        schema = update_array_schema(arg_types[0] if arg_types else origin, schema, docstring)

    return schema


def drop_titles(schema: dict[str, Any]) -> dict[str, Any]:
    """
    Remove title key-value pair from the entire schema. This helps in reducing token count.

    Args:
        schema: The schema to drop titles from

    Returns:
        The schema with titles dropped
    """
    schema.pop("title", None)

    if "anyOf" in schema:
        for sub_schema in schema["anyOf"]:
            drop_titles(sub_schema)

    elif "oneOf" in schema:
        for sub_schema in schema["oneOf"]:
            drop_titles(sub_schema)

    elif schema.get("type") == "object" and "properties" in schema:
        for prop in schema["properties"].values():
            drop_titles(prop)

    elif schema.get("type") == "array" and "items" in schema:
        drop_titles(schema["items"])

    return schema


def to_function_calling_format(name: str, schema: dict[str, Any], params_key: str) -> dict[str, Any]:
    """
    Transform to LLM function calling format

    Args:
        name:  The function name
        schema:  The function schema
        params_key:  The key to use for parameters

    Returns:
        The function calling formatted schema.
    """
    if schema.get("type") != "object":
        raise ValueError("Schema must be of object type.")

    description_dict: dict[str, str] = {}
    if desc := schema.pop("description", None):
        description_dict["description"] = desc

    return {
        "type": "function", "function": {
            "name": name, **description_dict, params_key: drop_titles(schema)
        }
    }


def to_openai_function_calling_format(name: str, schema: dict[str, Any]) -> dict[str, Any]:
    """
    Convert a function schema to the OpenAI function calling format.

    Args:
        name:  The name of the function.
        schema:  The function schema.

    Returns:
        Schema in OpenAI function calling format.
    """
    return to_function_calling_format(name, schema, "parameters")


def to_anthropic_function_calling_format(name: str, schema: dict[str, Any]) -> dict[str, Any]:
    """
    Convert a function schema to the Anthropic function calling format.

    Args:
        name:  The name of the function.
        schema:  The function schema.

    Returns:
        Schema in Anthropic function calling format.
    """
    return to_function_calling_format(name, schema, "input_schema")


def to_gemini_function_calling_format(name: str, schema: dict[str, Any]) -> dict[str, Any]:
    """
    Convert a function schema to the Gemini function calling format.

    Args:
        name:  The name of the function.
        schema:  The function schema.

    Returns:
        Schema in Gemini function calling format.
    """

    class SchemaModel(BaseModel):
        description: Optional[str] = None
        enum: Optional[list[str]] = None
        example: Optional[Any] = None
        format: Optional[str] = None
        nullable: Optional[bool] = None
        items: Optional["SchemaModel"] = None
        required: Optional[list[str]] = None
        type: str
        properties: Optional[dict[str, "SchemaModel"]] = None

    def add_enum_format(obj: dict[str, Any]) -> dict[str, Any]:
        if isinstance(obj, dict):
            new_dict: dict[str, Any] = {}
            for key, value in obj.items():
                new_dict[key] = add_enum_format(value)
                if key == "enum":
                    new_dict["format"] = "enum"
            return new_dict
        return obj

    schema_model = SchemaModel(**add_enum_format(schema))
    return to_function_calling_format(
        name=name,
        schema=schema_model.model_dump(exclude_none=True, exclude_unset=True),
        params_key="parameters",
    )

## Implementing a Json agent

Json Agent is very simple compared to other agents. It utilizes the function calling feature of LLMs to call relevant tool in an iterative manner until the final answer is obtained.

The agent processes query through the following loop:

- The agent receives a query and selects a relevant tool to use
- The tool is called and passed back to the LLM so it can decide whether to use "final_answer" tool or not
- If the tool is not "final_answer", the agent uses the tool's output as the input for the next tool
- If the tool is "final_answer", the agent returns the output as the final response to the given task

Json Agent is less transparent and less flexible compared to the ReAct agent.

In [76]:
# Setup Logger
import sys

from loguru import logger

logger.remove()

logger.add(sys.stdout, format="<level>{message}</level>")
logger = logger.opt(colors=True)

def highlight_role(role: str) -> str:
    return f"<bg #0f0707><fg #ffffff>[{role.upper()}]</fg #ffffff></bg #0f0707>"

In [77]:
from typing import Callable, Literal

from litellm import completion as get_completion
from litellm.utils import Message as AIMessage

                                
INSTRUCTION_TEMPLATE = """{description}
Complete the task by breaking it into smaller subtasks.
Use the "final_answer" tool only when you have completed all the subtasks.\
"""
"""Instruction template for Json agent."""


def final_answer(text: str) -> str:
    """Use to return final response if task has been completed or failed."""
    return text

class JsonAgent:
    """
    Json agent implementation that uses python functions to solve given tasks.
    The agent can be initialized with a list of functions or JsonTool instances.

    This agent uses litellm to connect with Language Models.
    Make sure to set the api key of the provider you are using.

    Examples:
        ```python
        # Define your functions or tools
        from duckduckgo_search import DDGS, exceptions

        def search_text(text: str):
            \"\"\"Search for text in the web.\"\"\"
            
            try:
                return DDGS().text(keywords=text)
            except exceptions.RatelimitException:
                return "Rate limit exceeded. Return final answer."

        # Initialize the agent
        agent = JsonAgent(search_text)

        # Call the agent
        agent(query="What is the meaning of life?", model="ollama/phi4")

        # See the final instruction
        agent.instruction

        # Take a peek at the history
        agent.show_history()
        ```
    """
    def __init__(
        self, 
        *functions_or_tools: Callable[P_Spec, T_Retval] | JsonTool[P_Spec, T_Retval],
        default_model: str,
        description: str | None = None,
    ) -> None:
        """
        Args:
            functions_or_tools: A list of functions or JsonTool instances.
            default_model: The default model to use for the agent. Explore models: https://docs.litellm.ai/docs/providers
            description: A description that tells what an agent is and is supposed to do.
        """

        if not functions_or_tools:
            raise ValueError("Atleast one function or tool is required.")
        
        self._tool_registry: dict[str, JsonTool] = {"final_answer": JsonTool(final_answer)}
        for entry in functions_or_tools:  # Update the registry with JsonTool instances
            tool = entry if isinstance(entry, JsonTool) else JsonTool(entry)
            if tool.name == "final_answer":
                continue
            self._tool_registry[tool.name] = tool

        self.default_model = default_model
        self.instruction = INSTRUCTION_TEMPLATE.format(description=description or "").strip()

        # Intialize message history with system instruction
        self.__message_history = [{"role": "system", "content": self.instruction}]

    def process_ai_message(self, message: AIMessage) -> bool:
        """Process an AI message and tell if the agent should continue solving the query."""

        if message.tool_calls:
            
            logger.info(highlight_role("self"))
            tool_call = message.tool_calls[0]
            self.__message_history.append({
                "role": "assistant",
                "tool_calls": [tool_call.to_dict(exclude_none=True)],
            })
            if (tool := self._tool_registry.get(tool_call.function.name)) is None:
                return True
            
            if tool_call.function.name != "final_answer":
                logger.info(f"⚙️ Tool: {tool_call.function.name!r}")
                logger.info(f"⌨️ Arguments: {tool_call.function.arguments!r}")

            tool_output = tool.call_with_raw_arguments(tool_call.function.arguments)
            if tool_call.function.name == "final_answer":
                self.__message_history.append({
                    "role": "assistant", "content": str(tool_output)
                })
                logger.info(f"📌 Final Answer: {tool_output}")
                return False
            
            if isinstance(tool_output, ExceptionInfo):
                logger.info(f"❌ {tool_output.name}: {tool_output.message}")
            else:
                logger.info(f"✅ Output: {str(tool_output)}")
            
            self.__message_history.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": str(tool_output),
            })
            return True
        else:
            if content := message.content.strip():
                logger.info(highlight_role("self"))
                logger.info(f"📝 Content: {content}")
                self.__message_history.append({"role": "assistant", "content": content})
                return True
        
        return False

    def __call__(
        self, 
        *, 
        query: str, 
        model: str | None = None, 
        tool_format: Literal["openai", "anthropic", "gemini"] = "openai",
    ) -> str:
        """
        Call the agent.

        Args:
            query: The query to complete
            model: The model to use in the place of the default model.
        """
        query = query.strip()
        logger.info(f"🖋️ Query: {query}\n")

        # Add user query to message history
        self.__message_history.append({"role": "user", "content": query})

        while True:
            # Get completion response from the model
            ai_message = get_completion(
                model or self.default_model, 
                self.__message_history, 
                tools=list(
                    getattr(tool, f"{tool_format}_schema", tool.openai_schema) 
                    for tool in self._tool_registry.values()
                ),
            ).choices[0].message
            
            # Process the AI message
            if not self.process_ai_message(ai_message):
                break
            
            logger.info("")
        
    @property
    def history(self) -> list[dict[str, str]]:
        """Messasge history"""
        return self.__message_history

    def show_history(self) -> None:
        """Display the message history."""
        import json
        for message in self.__message_history:
            if message["role"] == "assistant" and "tool_calls" in message:
                logger.info(f"{highlight_role(message['role'])}")
                for tool_call in message["tool_calls"]:
                    logger.info(json.dumps(tool_call, indent=4))
            else:
                logger.info(f"{highlight_role(message['role'])} \n{message['content']}\n")

    def clear_history(self) -> None:
        """Clear the message history."""
        self.__message_history.append({"role": "system", "content": self.instruction})


### Creating a Json agent

I am using Groq because it is super fast and has a good free usage limit. Get your own key from [here](https://console.groq.com/keys). If you want to use other models, you can select one from the [LiteLLM providers](https://docs.litellm.ai/docs/providers).

In [78]:
import os
from getpass import getpass

def set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"{var}: ")

set_env("GROQ_API_KEY")  # Replace with your LLM compatible environment key.

In [79]:
from dataclasses import dataclass
from typing import Any

import requests
from duckduckgo_search import DDGS

def search_text(text: str):
    """Search for text in the web."""
    
    return DDGS().text(keywords=text)

def fetch_json(url: str) -> dict[str, Any]:
    """Fetch JSON data from a given URL."""
    
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

@dataclass
class WeatherData:
    temperature: float
    wind_speed: float

    def __str__(self) -> str:
        return f"Current temperature: {self.temperature}°C, Current wind speed: {self.wind_speed} km/h"

def get_weather(city_name: str) -> WeatherData:
    """Get weather data for a given city."""

    json_response = fetch_json(
        f"https://geocoding-api.open-meteo.com/v1/search?name={city_name}"
    )

    if results := json_response.get("results", []):
        latitude, longitude = results[0]["latitude"], results[0]["longitude"]
    else:
        raise LookupError(f"City {city_name!r} not found.")
    
    current_weather = fetch_json(
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}&current_weather=true"
    ).get("current_weather", {})

    try:
        return WeatherData(
            temperature=current_weather["temperature"],
            wind_speed=current_weather["windspeed"],
        )
    except KeyError:
        raise LookupError("Weather data not available for city {city_name!r}.")

In [80]:
research_agent = JsonAgent(
    search_text, get_weather, 
    default_model="groq/llama-3.1-8b-instant",
    description="""\
You are a research agent who can find information and answer user queries.
"""
)
print(research_agent.instruction)

You are a research agent who can find information and answer user queries.

Complete the task by breaking it into smaller subtasks.
Use the "final_answer" tool only when you have completed all the subtasks.


In [81]:
research_agent(
    query="Based on the current weather of Kolkata, find the best place to visit."
)

🖋️ Query: Based on the current weather of Kolkata, find the best place to visit.

[SELF]
⚙️ Tool: 'get_weather'
⌨️ Arguments: '{"city_name": "Kolkata"}'
✅ Output: Current temperature: 25.9°C, Current wind speed: 8.9 km/h

[SELF]
⚙️ Tool: 'search_text'
⌨️ Arguments: '{"text": "best places to visit in Kolkata"}'
✅ Output: [{'title': 'THE 30 BEST Places to Visit in Kolkata (Calcutta) (2025) - Tripadvisor', 'href': 'https://www.tripadvisor.in/Attractions-g304558-Activities-Kolkata_Calcutta_Kolkata_District_West_Bengal.html', 'body': 'Science City Kolkata happens to be a remarkable hub of knowledge and innovation, offering an engaging experience for all ages. With its interactive exhibits, cutting-edge technology, and educational programs. One of the best places to indulge in Science in a fun way. Best place to bring kids and induce in them the love for Science.'}, {'title': '52 Best Places to visit in Kolkata | Top Tourist Attractions | 2025', 'href': 'https://www.holidify.com/places/kolka

In [82]:
research_agent.show_history()

[SYSTEM] 
You are a research agent who can find information and answer user queries.

Complete the task by breaking it into smaller subtasks.
Use the "final_answer" tool only when you have completed all the subtasks.

[USER] 
Based on the current weather of Kolkata, find the best place to visit.

[ASSISTANT]
{
    "function": {
        "arguments": "{\"city_name\": \"Kolkata\"}",
        "name": "get_weather"
    },
    "id": "call_f05z",
    "type": "function"
}
[TOOL] 
Current temperature: 25.9°C, Current wind speed: 8.9 km/h

[ASSISTANT]
{
    "function": {
        "arguments": "{\"text\": \"best places to visit in Kolkata\"}",
        "name": "search_text"
    },
    "id": "call_j411",
    "type": "function"
}
[TOOL] 
[{'title': 'THE 30 BEST Places to Visit in Kolkata (Calcutta) (2025) - Tripadvisor', 'href': 'https://www.tripadvisor.in/Attractions-g304558-Activities-Kolkata_Calcutta_Kolkata_District_West_Bengal.html', 'body': 'Science City Kolkata happens to be a remarkable hub of